In [95]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
np.set_printoptions(suppress=True)

In [105]:
DOUBT, ACCEPT = 0, 1
SIDES = 6

In [78]:
class Node():
    def __init__(self, num_actions):
        self.strategy = np.zeros(num_actions)
        self.strategy_sum = np.zeros(num_actions)
        self.regret_sum = np.zeros(num_actions)
        self.utility = 0
        self.player_visit_prob_sum = 0
        self.opponent_visit_prob_sum = 0
    
    def normalize(self, value):
        normalizing_sum = np.sum(value)
        if normalizing_sum > 0:
            self.strategy = value / normalizing_sum
        else:
            self.strategy = np.ones(value.shape[0]) / value.shape[0]
        return self.strategy
    
    def get_strategy(self):
        return self.normalize(np.maximum(self.regret_sum, 0))

    def get_average_strategy(self):
        return self.normalize(self.strategy_sum)

In [214]:
def train(iterations):
    response_nodes = np.empty((SIDES, SIDES+1), dtype=type(Node))    
    # The previous claim (player_claim) cannot have claimed the highest value die as the opponent would have not choice but to DOUBT
    # And therefore will never reach this state 
    for player_claim in range(SIDES):
        # The next player, on the other hand, can claim the highest value die, hence the range is to SIDES+1 instead of SIDES
        for opponent_claim in range(player_claim + 1, SIDES + 1):
            # If the opponent claims the highest value die then there is only one action left at this node: DOUBT
            num_actions = 1 if opponent_claim == SIDES else 2
            response_nodes[player_claim, opponent_claim] = Node(num_actions)

    claim_nodes = np.empty((SIDES, SIDES+1), dtype=type(Node))
    for opponent_claim in range(SIDES):
        for roll in range(1, SIDES + 1):
            claim_nodes[opponent_claim, roll] = Node(SIDES - opponent_claim)

    regret = np.zeros(SIDES)
    for i in trange(iterations):
        roll_after_accepting_claim = np.random.randint(1, SIDES+1, SIDES)
        claim_nodes[0][roll_after_accepting_claim[0]].player_visit_prob_sum = 1
        claim_nodes[0][roll_after_accepting_claim[0]].opponent_visit_prob_sum = 1

        # Propagate forward
        for opponent_claim in range(SIDES+1):
            # Visit response nodes forward
            for player_claim in range(opponent_claim):
                node: Node = response_nodes[player_claim, opponent_claim]
                strategy = node.get_strategy()
                node.strategy_sum += strategy * node.player_visit_prob_sum
                if opponent_claim < SIDES:
                    next_node: Node = claim_nodes[opponent_claim, roll_after_accepting_claim[opponent_claim]]
                    next_node.player_visit_prob_sum += strategy[1] * node.player_visit_prob_sum
                    next_node.opponent_visit_prob_sum += node.opponent_visit_prob_sum

            # Visit claim nodes forward
            if opponent_claim < SIDES:
                node: Node = claim_nodes[opponent_claim, roll_after_accepting_claim[opponent_claim]]
                strategy = node.get_strategy()
                node.strategy_sum += strategy * node.player_visit_prob_sum
                for player_claim in range(opponent_claim + 1, SIDES + 1):
                    next_claim_probability = strategy[player_claim - opponent_claim - 1]
                    if next_claim_probability > 0: 
                        next_node: Node = response_nodes[opponent_claim, player_claim]
                        next_node.player_visit_prob_sum += node.opponent_visit_prob_sum
                        next_node.opponent_visit_prob_sum += next_claim_probability * node.player_visit_prob_sum

        # Propagate backwards
        for opponent_claim in range(SIDES, -1, -1):
            # Visit claim nodes backwards
            if opponent_claim < SIDES:
                node: Node = claim_nodes[opponent_claim, roll_after_accepting_claim[opponent_claim]]
                strategy = node.strategy
                node.strategy_sum += strategy * node.player_visit_prob_sum
                node.utility = 0

                for player_claim in range(opponent_claim + 1, SIDES + 1):
                    action_index = player_claim - opponent_claim - 1
                    next_node: Node = response_nodes[opponent_claim, player_claim]
                    child_utility = -next_node.utility
                    regret[action_index] = child_utility
                    node.utility += strategy[action_index] * child_utility
                
                regret -= node.utility
                node.regret_sum += regret[:node.regret_sum.shape[0]] * node.opponent_visit_prob_sum

                node.player_visit_prob_sum = node.opponent_visit_prob_sum = 0

            # Visit response nodes backwards
            for player_claim in range(opponent_claim):
                node: Node = response_nodes[player_claim, opponent_claim]
                strategy = node.strategy
                doubt_utility = 1 if opponent_claim > roll_after_accepting_claim[player_claim] else -1
                regret[DOUBT] = doubt_utility
                node.utility = strategy[DOUBT] * doubt_utility
                if opponent_claim < SIDES:
                    next_node: Node = claim_nodes[opponent_claim, roll_after_accepting_claim[opponent_claim]]
                    regret[ACCEPT] = next_node.utility
                    node.utility += strategy[ACCEPT] * next_node.utility

                regret -= node.utility
                node.regret_sum += regret[:node.regret_sum.shape[0]] * node.opponent_visit_prob_sum

                node.player_visit_prob_sum = node.opponent_visit_prob_sum = 0

        if i == iterations // 2:
            for nodes in response_nodes:
                for node in nodes:
                    if node is not None:
                        node.strategy_sum[:] = 0
            for nodes in claim_nodes:
                for node in nodes:
                    if node is not None:
                        node.strategy_sum[:] = 0

    print("\nPrevious Claim\tNew Claim\tDoubt / Accept")
    for player_claim in range(SIDES + 1):
        for opponent_claim in range(player_claim + 1, SIDES + 1):
            print(player_claim, '\t\t', opponent_claim, '\t\t', response_nodes[player_claim, opponent_claim].get_average_strategy())

    print("\nPrevious Claim\tRoll\tAction Probabilities")
    for opponent_claim in range(SIDES):
        for roll in range(1, SIDES + 1):
            print(opponent_claim, '\t\t', roll, '\t\t', claim_nodes[opponent_claim, roll].get_average_strategy())

In [216]:
train(5000)

100%|██████████| 5000/5000 [00:03<00:00, 1419.24it/s]


Previous Claim	New Claim	Doubt / Accept
0 		 1 		 [0. 1.]
0 		 2 		 [0. 1.]
0 		 3 		 [0.29351523 0.70648477]
0 		 4 		 [0.44256014 0.55743986]
0 		 5 		 [0.55413591 0.44586409]
0 		 6 		 [1.]
1 		 2 		 [0.5 0.5]
1 		 3 		 [0.92187671 0.07812329]
1 		 4 		 [0.34946587 0.65053413]
1 		 5 		 [0.41879085 0.58120915]
1 		 6 		 [1.]
2 		 3 		 [0.27312187 0.72687813]
2 		 4 		 [1. 0.]
2 		 5 		 [0.72916221 0.27083779]
2 		 6 		 [1.]
3 		 4 		 [1. 0.]
3 		 5 		 [1. 0.]
3 		 6 		 [1.]
4 		 5 		 [1. 0.]
4 		 6 		 [1.]
5 		 6 		 [1.]

Previous Claim	Roll	Action Probabilities
0 		 1 		 [0.0979273  0.05823683 0.72624077 0.00253567 0.11505944 0.        ]
0 		 2 		 [0.00894643 0.07127295 0.221137   0.56134396 0.13729966 0.        ]
0 		 3 		 [0. 0. 1. 0. 0. 0.]
0 		 4 		 [0. 0. 0. 1. 0. 0.]
0 		 5 		 [0. 0. 0. 0. 1. 0.]
0 		 6 		 [0. 0. 0. 0. 0. 1.]
1 		 1 		 [0.         0.80105569 0.         0.19894431 0.        ]
1 		 2 		 [0.         0.58673928 0.41326072 0.         0.        ]
1 		 3 		 [0. 1. 

In [211]:
response_nodes1 = np.zeros((SIDES, SIDES+1))
response_nodes1[:,1:] = [[i for _ in range(1, SIDES + 1)] for i in range(SIDES, 0, -1)]
print(response_nodes1)
claim_nodes = np.zeros((SIDES, SIDES+1))
for opponent_claim in range(SIDES):
       for roll in range(1, SIDES + 1):
            claim_nodes[opponent_claim, roll] = SIDES - opponent_claim
print(claim_nodes)
print(claim_nodes == response_nodes1)

response_nodes = np.empty((SIDES, SIDES+1), dtype=type(Node))
response_nodes[np.triu_indices(SIDES, 1)] = [Node(2) for _ in range(np.triu_indices(SIDES, 1)[0].size)]
print(response_nodes)
print(np.triu_indices(SIDES, 1))

[[0. 6. 6. 6. 6. 6. 6.]
 [0. 5. 5. 5. 5. 5. 5.]
 [0. 4. 4. 4. 4. 4. 4.]
 [0. 3. 3. 3. 3. 3. 3.]
 [0. 2. 2. 2. 2. 2. 2.]
 [0. 1. 1. 1. 1. 1. 1.]]
[[0. 6. 6. 6. 6. 6. 6.]
 [0. 5. 5. 5. 5. 5. 5.]
 [0. 4. 4. 4. 4. 4. 4.]
 [0. 3. 3. 3. 3. 3. 3.]
 [0. 2. 2. 2. 2. 2. 2.]
 [0. 1. 1. 1. 1. 1. 1.]]
[[ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]]
[[None <__main__.Node object at 0x00000277DA439950>
  <__main__.Node object at 0x00000277DA2A6050> None]
 [None None <__main__.Node object at 0x00000277DA2A80D0>
  <__main__.Node object at 0x00000277DA2AA9D0> None]
 [None None None <__main__.Node object at 0x00000277DA2A53D0>
  <__main__.Node object at 0x00000277DA2A9710> None]
 [None None None None <__main__.Node object at 0x00000277DA2AB9D0>
  <__main__.Node object at 0x00000277DA357